<a href="https://colab.research.google.com/github/abdulkadirdemirci/RFM-analysis/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

################################
# iş problemi
################################

"""
Online ayakkabı mağazası olan FLO müşterilerini
segmentlere ayırıp bu segmentlere göre pazarlama
stratejileri belirlemek istiyor. Buna yönelik olarak
müşterilerin davranışları tanımlanacak ve bu
davranışlardaki öbeklenmelere göre gruplar oluşturulacak.
"""

################################
# veri seti hikayesi
################################

"""
Veri seti Flo’dan son alışverişlerini 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan)
olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

master_id          -Eşsiz müşteri numarası
order_channel      -Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile)
last_order_channel -En son alışverişin yapıldığı kanal
first_order_date   -Müşterinin yaptığı ilk alışveriş tarihi
last_order_date    -Müşterinin yaptığı son alışveriş tarihi
last_order_date_online       -Müşterinin online platformda yaptığı son alışveriş tarihi
last_order_date_offline      -Müşterinin offline platformda yaptığı son alışveriş tarihi
order_num_total_ever_online  -Müşterinin online platformda yaptığı toplam alışveriş sayısı
order_num_total_ever_offline -Müşterinin offline'da yaptığı toplam alışveriş sayısı
customer_value_total_ever_offline  -Müşterinin offline alışverişlerinde ödediği toplam ücret
customer_value_total_ever_online   -Müşterinin online alışverişlerinde ödediği toplam ücret
interested_in_categories_12        -Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi
"""

################################
# GÖREV1 : VERİYİ ANLAMA VE HAZIRLAMA
################################

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

################################
# GÖREV1 - ADIM1 : flo_data_20K.csv verisini okuyunuz.Dataframe’in kopyasını oluşturunuz.
################################

In [2]:
df_ = pd.read_csv("/content/drive/MyDrive/flo_data_20k.csv")
df = df_.copy()

################################
# GÖREV1 - ADIM2 : Veri setinde
# a. İlk 10 gözlem,
# b. Değişken isimleri,
# c. Betimsel istatistik,
# d. Boş değer,
# e. Değişken tipleri, incelemesi yapınız.
################################

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [4]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [5]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13


In [7]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

################################
# GÖREV1 - ADIM3 : Omnichannel müşterilerin hem online'dan hemde offline
# platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
# alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz
################################

In [9]:
df["all_over_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["all_over_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]


################################
# GÖREV1 - ADIM4 :  Değişken tiplerini inceleyiniz.
# Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
################################

In [10]:
from datetime import datetime

In [12]:
df["last_order_date"] = pd.to_datetime(df["last_order_date"])
df["first_order_date"] = pd.to_datetime(df["first_order_date"])
df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"])
df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"])

################################
# GÖREV1 - ADIM5 :  Alışveriş kanallarındaki müşteri sayısının,
# toplam alınan ürün sayısının ve
# toplam harcamaların dağılımına bakınız.
################################

In [13]:
df["order_channel"].nunique()

4

In [14]:
df["order_channel"].value_counts()

Android App    9495
Mobile         4882
Ios App        2833
Desktop        2735
Name: order_channel, dtype: int64

In [15]:
df.groupby("order_channel").agg({'all_over_order': ["sum"],
                                 'all_over_value': ["sum"],
                                 "order_channel": ["count"]}
                                )

,all_over_order,all_over_value,order_channel
,sum,sum,count
order_channel,,,
Android App,52269.0,7819062.76,9495
Desktop,10920.0,1610321.46,2735
Ios App,15351.0,2525999.93,2833
Mobile,21679.0,3028183.16,4882


################################
# GÖREV1 - ADIM6 :  En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.
################################

In [16]:
df.sort_values(by="all_over_value", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0,1.0,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.0,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0,1.0,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0,1.0,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.0,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0,3.0,18119.14,2587.20,[AKTIFSPOR],4.0,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0,4.0,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.0,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0,1.0,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.0,1.0,180.73,12545.37,"[ERKEK, AKTIFSPOR]",37.0,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0,1.0,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.0,2.0,64.97,12038.18,"[ERKEK, KADIN]",20.0,12103.15


################################
# GÖREV1 - ADIM7 :  En fazla siparişi veren ilk 10 müşteriyi sıralayınız.
################################

In [17]:
df.sort_values(by="all_over_order", ascending=False).head(10)


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0,1.0,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0,109.0,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.0,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0,87.0,8432.25,139.98,[ERKEK],88.0,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0,81.0,3997.55,242.81,[ERKEK],83.0,4240.36
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0,1.0,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0,33918.10
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.0,74.0,4640.77,543.28,"[KADIN, AKTIFSPOR]",77.0,5184.05
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.0,3.0,292.93,5004.95,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.0,5297.88
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0,1.0,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0,16918.57
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0,4.0,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.0,18443.57


################################
# GÖREV1 - ADIM8 : Veri ön hazırlık sürecini fonksiyonlaştırınız.
################################

In [19]:
def hazirlik(df, plot=False):
    print("*" * 50)
    print("genel bakış")
    print("*" * 50)
    df.head()
    df.shape
    df.columns
    df.describe().T
    df.isnull().sum()
    df.info()
    print("\n\n")
    print("*" * 50)
    print("birleştirilen değerler")
    print("*" * 50)
    df["all_over_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["all_over_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
    print("birleştirmeler tamamlandı....")
    print("*" * 50)
    print("gereken degişkenleri date e çevirmek")
    print("*" * 50)
    from datetime import date
    df = df.astype({"first_order_date": "datetime64[ns]"})
    df = df.astype({"last_order_date": "datetime64[ns]"})
    df = df.astype({"last_order_date_online": "datetime64[ns]"})
    df = df.astype({"last_order_date_offline": "datetime64[ns]"})
    print(df.info())
    print("*" * 50)
    print("groupby altında incelemeler")
    print("*" * 50)
    print(df.groupby("order_channel").agg({'all_over_order': ["sum"],
                                           'all_over_value': ["sum"],
                                           "order_channel": ["count"]})
          )
    print("*" * 50)
    print("en fazla kazanc getiren ilk 10 müşteri")
    print("*" * 50)
    print(df.sort_values(by="all_over_value", ascending=False).head(10))
    print("*" * 50)
    print("en fazla alışveriş yapan ilk 10 müşteri")
    print("*" * 50)
    print(df.sort_values(by="all_over_order", ascending=False).head(10))
    if plot:
        plt.pie(x=df["order_channel"].value_counts().values.tolist(),
                labels=df["order_channel"].value_counts().keys().tolist(),
                colors=["#ffdb00", "#ffa904", "#ee7b06", "#a12424"],
                autopct="%1.1f%%")
    plt.show()

In [20]:
hazirlik(df, plot=False)

**************************************************
genel bakış
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64    

################################
# GÖREV2 : RFM METRİKLERİNİN HESAPLANMASI
################################
################################
# GÖREV2 - ADIM1: : Recency, Frequency ve Monetary tanımlarını yapınız.
################################

"""
Recency :
Yenilik olarak da tanımlanabilir. Birimi CLTV'nün aksine gündür.
Son alışverişin analiz tarihinden kaç gün önce yapıldığının bir ölçüsüdür.
[analiz_tarihi]-[son_alışveriş_tarihi] formülü ile bulunur.

Frequency :
Belirlenen bir zaman dilimin den başlayarak analiz tarihine kadar
müşterinin kaç kez alış veriş yaptığının bir göstergesidir.
Analizin periyoduna bağlı olarak aylık, 2 haftalık, 3 aylık gibi zaman dilimlerinde
müşteri frekansı değerlendirilebilir.

Monetary :
Müşterinin belirli zaman dilimi içerisinde yaptığı alışverişler 
neticesinde kuruma/şirkete kazandırdığı değerin parasal karşılığı.
Geçerli zaman dilimi içerisinde müşterinin aldığı ürünlerin fiyatları toplanarak elde edilir
"""

################################
# GÖREV2 - ADIM2-3- : Müşteri özelinde Recency, Frequency ve Monetary metriklerini hesaplayınız.
# Hesapladığınız metrikleri rfm isimli bir değişkene atayınız.
# Oluşturduğunuz metriklerin isimlerini recency, frequency ve monetary olarak değiştiriniz.
################################

In [21]:
df.shape

(19945, 14)

veri setimiz 19945 gözlem ve 14 değişkenden oluşmaktadır.
 df["master_id"].nunique() 19945 den az çıkarsa çoklamalar vardır anlamına gelir

In [22]:
df["master_id"].nunique()

19945

 çoklama yoktur her müşteri için sadece bir adet gözlem birimi vardır.
veri setindeki en son alışveriş tarihini bul!

In [23]:
import datetime as dt
from datetime import datetime

In [24]:
df["last_order_date"] = pd.to_datetime(df["last_order_date"])
df["first_order_date"] = pd.to_datetime(df["first_order_date"])
df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"])
df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [26]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [30]:
today = dt.datetime(2021, 6, 1)

In [31]:
df["recency"] = df.apply(lambda row: (today - row["last_order_date"]).days, axis=1)
df["frequency"] = df["all_over_order"]
df["monetary"] = df["all_over_value"]

In [32]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,95,5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,105,21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,186,5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,135,2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,86,2.0,209.98


################################
# GÖREV3: RF Skorunun Hesaplanması
################################
################################
# GÖREV3 - ADIM1-2 : Recency, Frequency ve Monetary metriklerini qcut yardımı ile 1-5 arasında skorlara çeviriniz.
# Bu skorları recency_score, frequency_score ve monetary_score olarak kaydediniz
################################

In [33]:
df["recency_score"] = pd.qcut(df["recency"], 5, labels=[5, 4, 3, 2, 1])
df["frequency_score"] = pd.qcut(df["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
df["monetary_score"] = pd.qcut(df["monetary"], 5, labels=[1, 2, 3, 4, 5])
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,95,5.0,939.37,3,4,4
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,105,21.0,2013.55,3,5,5
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,186,5.0,585.32,2,4,3
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,135,2.0,121.97,3,1,1
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,86,2.0,209.98,3,1,1


################################
# GÖREV3 - ADIM3 : recency_score ve frequency_score’u tek bir değişken olarak ifade ediniz ve RF_SCORE olarak kaydediniz.
################################

In [34]:
df["RF_Score"] = df.apply(lambda row: "%s%s" % (row["recency_score"], row["frequency_score"]), axis=1)

In [35]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,95,5.0,939.37,3,4,4,34
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,105,21.0,2013.55,3,5,5,35
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,186,5.0,585.32,2,4,3,24
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,135,2.0,121.97,3,1,1,31
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,86,2.0,209.98,3,1,1,31


################################
# GÖREV4 : RF Skorunun Segment Olarak Tanımlanması
################################
################################
# GÖREV4 - ADIM1 : Oluşturulan RF skorları için segment tanımlamaları yapınız.
################################

In [38]:
seg_map = {r"[1-2][5]": "cant_loose_them",
           r"[3-4][4-5]": "loyal_customers",
           r"[3][3]": "need_attention",
           r"[1-2][3-4]": "at_rist",
           r"[1-2][1-2]": "hibernating",
           r"[4-5][2-3]": "potential_royalist",
           r"[5][1]": "new_customers",
           r"[5][4-5]": "champions",
           r"[3][1-2]": "about_to_sleep",
           r"[4][1]": "promising"
           }

################################
# GÖREV4 - ADIM2 :  seg_map yardımı ile skorları segmentlere çeviriniz.
################################

In [36]:
df["Segment"] = df["RF_Score"]

In [39]:
df["Segment"] = df["Segment"].replace(seg_map, regex=True)

In [40]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score,Segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37,95,5.0,939.37,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55,105,21.0,2013.55,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32,186,5.0,585.32,2,4,3,24,at_rist
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,135,2.0,121.97,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98,86,2.0,209.98,3,1,1,31,about_to_sleep


################################
# GÖREV5 :  Aksiyon Zamanı !
################################
################################
# GÖREV5 - ADIM1 :  Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.
################################

In [42]:
df.groupby("Segment").agg({"recency": ["mean", "count"],
                            "frequency": ["mean", "count"],
                            "monetary": ["mean", "count"]})

recency        frequency           monetary      
                          mean count       mean count         mean count
Segment                                                                 
about_to_sleep      113.785144  1629   2.401473  1629   359.008963  1629
at_rist             241.606835  3131   4.472373  3131   646.610236  3131
cant_loose_them     235.444167  1200  10.698333  1200  1474.468217  1200
champions            17.106625  1932   8.934265  1932  1406.625109  1932
hibernating         247.949501  3604   2.394007  3604   366.267062  3604
loyal_customers      82.594763  3361   8.374591  3361  1216.818616  3361
need_attention      113.828676   823   3.727825   823   562.142965   823
new_customers        17.917647   680   2.000000   680   339.955544   680
potential_royalist   37.155888  2938   3.304289  2938   533.184466  2938
promising            58.921175   647   2.000000   647   335.672705   647

################################
# GÖREV5 - ADIM2-a :
# FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri
# tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak
# iletişime geçmek isteniliyor. Sadık müşterilerinden(champions, loyal_customers) ve kadın kategorisinden alışveriş
# yapan kişiler özel olarak iletişim kurulacak müşteriler. Bu müşterilerin id numaralarını csv dosyasına kaydediniz.
################################

In [43]:
df.loc[(df["interested_in_categories_12"].str.contains("KADIN")) & (
            (df["Segment"] == "loyal_customers") | (df["Segment"] == "champions"))]["master_id"]

0        cc294636-19f0-11eb-8d74-000d3a38a36f
1        f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8        cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11       c2e15af2-9eed-11e9-9897-000d3a38a36f
33       fb840306-1219-11ea-a001-000d3a38a36f
                         ...                 
19912    f63aba0e-41d8-11ea-96d9-000d3a38a36f
19913    81d0da04-a53e-11e9-a2fc-000d3a38a36f
19914    825e6b00-ab40-11e9-a2fc-000d3a38a36f
19917    f8c471c8-2596-11eb-81e9-000d3a38a36f
19926    7e1c15be-6298-11ea-9861-000d3a38a36f
Name: master_id, Length: 2497, dtype: object

################################
# GÖREV5 - ADIM2-b :
#  Erkek ve Çocuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte
# iyi müşteri olan ama uzun süredir alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni
# gelen müşteriler özel olarak hedef alınmak isteniyor. Uygun profildeki müşterilerin id'lerini csv dosyasına kaydediniz.
################################

In [44]:
df.loc[((df["interested_in_categories_12"].str.contains("ERKEK")) |
        (df["interested_in_categories_12"].str.contains("COCUK"))) &
       ((df["Segment"] == "cant_loose_them") | (df["Segment"] == "about_to_sleep") | (df["Segment"] == "new_customers"))]["master_id"]


3        1854e56c-491f-11eb-806e-000d3a38a36f
15       13ed97a4-b167-11e9-89fa-000d3a38a36f
23       4fbb0cbe-9085-11ea-af25-000d3a38a36f
34       1acf7870-87af-11ea-ace9-000d3a38a36f
56       92a334ee-58f2-11eb-9e65-000d3a38a36f
                         ...                 
19888    36c10134-9ea3-11e9-9897-000d3a38a36f
19932    13a5e98c-a824-11e9-a2fc-000d3a38a36f
19933    6d285c64-5e73-11ea-be5b-000d3a38a36f
19935    b514d468-5b52-11ea-85ca-000d3a38a36f
19936    1982ac0e-9f4c-11e9-9897-000d3a38a36f
Name: master_id, Length: 1759, dtype: object

In [47]:
df.loc[((df["interested_in_categories_12"].str.contains("ERKEK")) |
        (df["interested_in_categories_12"].str.contains("COCUK"))) &
       ((df["Segment"] == "cant_loose_them") | (df["Segment"] == "about_to_sleep") | (df["Segment"] == "new_customers"))]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score,Segment
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97,135,2.0,121.97,3,1,1,31,about_to_sleep
15,13ed97a4-b167-11e9-89fa-000d3a38a36f,Mobile,Desktop,2019-06-13,2020-08-17,2020-08-17,2020-07-28,7.0,1.0,112.97,819.39,"[AKTIFCOCUK, COCUK, KADIN]",8.0,932.36,288,8.0,932.36,1,5,4,15,cant_loose_them
23,4fbb0cbe-9085-11ea-af25-000d3a38a36f,Android App,Offline,2020-05-11,2021-01-29,2020-05-11,2021-01-29,1.0,1.0,172.97,69.48,"[COCUK, KADIN]",2.0,242.45,123,2.0,242.45,3,1,1,31,about_to_sleep
34,1acf7870-87af-11ea-ace9-000d3a38a36f,Ios App,Offline,2020-04-26,2021-02-18,2020-04-26,2021-02-18,1.0,1.0,160.97,137.12,"[ERKEK, KADIN]",2.0,298.09,103,2.0,298.09,3,1,1,31,about_to_sleep
56,92a334ee-58f2-11eb-9e65-000d3a38a36f,Android App,Android App,2021-01-28,2021-02-11,2021-02-11,2021-01-28,1.0,1.0,119.87,94.99,"[ERKEK, KADIN]",2.0,214.86,110,2.0,214.86,3,1,1,31,about_to_sleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,36c10134-9ea3-11e9-9897-000d3a38a36f,Android App,Offline,2019-03-06,2020-06-29,2020-06-19,2020-06-29,2.0,7.0,859.87,111.98,"[AKTIFCOCUK, COCUK]",9.0,971.85,337,9.0,971.85,1,5,4,15,cant_loose_them
19932,13a5e98c-a824-11e9-a2fc-000d3a38a36f,Android App,Offline,2015-03-23,2020-11-05,2020-06-28,2020-11-05,7.0,2.0,210.72,963.28,"[ERKEK, KADIN, AKTIFSPOR]",9.0,1174.00,208,9.0,1174.00,2,5,5,25,cant_loose_them
19933,6d285c64-5e73-11ea-be5b-000d3a38a36f,Android App,Android App,2020-01-15,2020-10-24,2020-10-24,2020-02-28,7.0,1.0,79.99,1034.81,"[AKTIFCOCUK, ERKEK, COCUK]",8.0,1114.80,220,8.0,1114.80,2,5,5,25,cant_loose_them
19935,b514d468-5b52-11ea-85ca-000d3a38a36f,Android App,Android App,2020-01-03,2021-02-16,2021-02-16,2020-01-03,1.0,1.0,119.99,140.00,[ERKEK],2.0,259.99,105,2.0,259.99,3,2,1,32,about_to_sleep


In [45]:
df.loc[((df["interested_in_categories_12"].str.contains("ERKEK")) | (
    df["interested_in_categories_12"].str.contains("COCUK"))) & (
            (df["Segment"] == "cant_loose_them") | (df["Segment"] == "aboout_to_sleep") | (
                df["Segment"] == "new_customers"))]["master_id"]

15       13ed97a4-b167-11e9-89fa-000d3a38a36f
76       7d58deb6-62fa-11ea-a6dc-000d3a38a36f
91       808005da-a511-11e9-a2fc-000d3a38a36f
147      90e0b158-2b54-11ea-9d27-000d3a38a36f
166      9613613c-c9d0-11ea-a31e-000d3a38a36f
                         ...                 
19868    6a138f62-d71e-11e9-93bc-000d3a38a36f
19888    36c10134-9ea3-11e9-9897-000d3a38a36f
19932    13a5e98c-a824-11e9-a2fc-000d3a38a36f
19933    6d285c64-5e73-11ea-be5b-000d3a38a36f
19936    1982ac0e-9f4c-11e9-9897-000d3a38a36f
Name: master_id, Length: 994, dtype: object

In [46]:
df.loc[((df["interested_in_categories_12"].str.contains("ERKEK")) | (
    df["interested_in_categories_12"].str.contains("COCUK"))) & (
            (df["Segment"] == "cant_loose_them") | (df["Segment"] == "aboout_to_sleep") | (
                df["Segment"] == "new_customers"))]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,all_over_order,all_over_value,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score,Segment
15,13ed97a4-b167-11e9-89fa-000d3a38a36f,Mobile,Desktop,2019-06-13,2020-08-17,2020-08-17,2020-07-28,7.0,1.0,112.97,819.39,"[AKTIFCOCUK, COCUK, KADIN]",8.0,932.36,288,8.0,932.36,1,5,4,15,cant_loose_them
76,7d58deb6-62fa-11ea-a6dc-000d3a38a36f,Mobile,Mobile,2016-06-14,2020-11-23,2020-11-23,2019-11-16,7.0,1.0,139.99,1231.46,"[AKTIFCOCUK, COCUK, KADIN, AKTIFSPOR]",8.0,1371.45,190,8.0,1371.45,2,5,5,25,cant_loose_them
91,808005da-a511-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-06-21,2020-10-29,2020-10-29,2019-11-16,6.0,2.0,268.97,1029.87,"[COCUK, KADIN]",8.0,1298.84,215,8.0,1298.84,2,5,5,25,cant_loose_them
147,90e0b158-2b54-11ea-9d27-000d3a38a36f,Android App,Android App,2019-11-30,2020-10-16,2020-10-16,2020-01-26,7.0,1.0,39.99,2214.45,"[AKTIFCOCUK, ERKEK, COCUK, AKTIFSPOR]",8.0,2254.44,228,8.0,2254.44,2,5,5,25,cant_loose_them
166,9613613c-c9d0-11ea-a31e-000d3a38a36f,Ios App,Ios App,2020-07-19,2021-05-30,2021-05-30,2020-07-19,1.0,1.0,129.99,360.98,[ERKEK],2.0,490.97,2,2.0,490.97,5,1,3,51,new_customers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19868,6a138f62-d71e-11e9-93bc-000d3a38a36f,Mobile,Mobile,2019-09-14,2020-12-06,2020-12-06,2019-09-14,6.0,1.0,29.99,991.27,"[AKTIFCOCUK, COCUK, KADIN, AKTIFSPOR]",7.0,1021.26,177,7.0,1021.26,2,5,5,25,cant_loose_them
19888,36c10134-9ea3-11e9-9897-000d3a38a36f,Android App,Offline,2019-03-06,2020-06-29,2020-06-19,2020-06-29,2.0,7.0,859.87,111.98,"[AKTIFCOCUK, COCUK]",9.0,971.85,337,9.0,971.85,1,5,4,15,cant_loose_them
19932,13a5e98c-a824-11e9-a2fc-000d3a38a36f,Android App,Offline,2015-03-23,2020-11-05,2020-06-28,2020-11-05,7.0,2.0,210.72,963.28,"[ERKEK, KADIN, AKTIFSPOR]",9.0,1174.00,208,9.0,1174.00,2,5,5,25,cant_loose_them
19933,6d285c64-5e73-11ea-be5b-000d3a38a36f,Android App,Android App,2020-01-15,2020-10-24,2020-10-24,2020-02-28,7.0,1.0,79.99,1034.81,"[AKTIFCOCUK, ERKEK, COCUK]",8.0,1114.80,220,8.0,1114.80,2,5,5,25,cant_loose_them
